<a href="https://colab.research.google.com/github/bthodla/danano/blob/master/prj4/wrangle_act.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Gathering data

1.   Load the "twitter-archive-enhanced.csv" into a dataframe
2.   Load the "image_predictions.tsv" (the Tweet image predictions) hosted on Udacity servers programmatically using the "requests" library (URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv)
3. Using tweet ids from (1) above and the Twitter API, load information such as "retweet count" and "favorite count" and any other interesting data items and store the entire set of data relating to a tweet in a JSON file called "tweet_json.txt"; then read this file line by line into a data frame and include tweet id, retweet count, favorite count and any other data items of interest
4. Consumer API keys: 
    
    Z22dLMPEOtCL2ayqnPTbFg3sK (API key)
    
    SuaxZay016BTIpzrYszo9Dbo0jQd38FZ8SUi0bDyy8hU6jMKdj (API secret key)
    
    Access token & access token secret

    14299634-DHvEoZI9bR2D5WOZWf82MjHX6vEnPRIosIrli6ueb (Access token)

    nvba5wXGFrEcYKmQjztu40wsr0vG2pXnQShE2ZQv6I313 (Access token secret)

    Read and write (Access level)




In [0]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import requests
import io
import tweepy
import json
from google.colab import drive
import yaml
import time
import os


In [0]:
def load_yaml_file(filename: str) -> dict:
    twitter_keys = {}

    with open(filename, 'r') as ymlfile:
        cfg = yaml.safe_load(ymlfile)
    
    twitter_keys['consumer_key'] = cfg['twitter']['consumer']['key']
    twitter_keys['consumer_secret'] = cfg['twitter']['consumer']['secret']
    twitter_keys['access_token'] = cfg['twitter']['access']['token']
    twitter_keys['access_token_secret'] = cfg['twitter']['access']['token-secret']

    return twitter_keys

In [0]:
def print_file(file_name, lines = 10):
    with open(file_name, 'r') as f:
        for _ in range(lines):
            print (f.readline())
        

In [24]:
drive.mount('/content/gdrive')
twitter_keys_yaml_file = '/content/gdrive/My Drive/Colab Notebooks/twitter_keys.yaml'
tweet_json_file = '/content/gdrive/My Drive/Colab Notebooks/tweet_json.txt'

"""
with open(twitter_keys_yaml_file, encoding='utf-8') as file:
    line = file.readline()
    while line:
        line = file.readline()
        print(line)
"""
print(load_yaml_file(twitter_keys_yaml_file))


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
{'consumer_key': 'Z22dLMPEOtCL2ayqnPTbFg3sK', 'consumer_secret': 'SuaxZay016BTIpzrYszo9Dbo0jQd38FZ8SUi0bDyy8hU6jMKdj', 'access_token': '14299634-DHvEoZI9bR2D5WOZWf82MjHX6vEnPRIosIrli6ueb', 'access_token_secret': 'nvba5wXGFrEcYKmQjztu40wsr0vG2pXnQShE2ZQv6I313'}


In [25]:
# 1. Load the file "twitter-archive-enhanced.csv" into a dataframe
twitter_archive_file_url = 'https://raw.githubusercontent.com/bthodla/danano/master/prj4/twitter-archive-enhanced.csv'

tweets_df = pd.read_csv(twitter_archive_file_url)

tweet_count = tweets_df.shape[0]
print ('%s %d' % ('Tweet Count: ', tweet_count))

Tweet Count:  2356


In [0]:
# 2. Load the "image_predictions.tsv" (the Tweet image predictions) hosted on Udacity servers programmatically using the "requests" library

image_predictions_file_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

url_data = requests.get(image_predictions_file_url).content

img_pred_df = pd.read_csv(io.StringIO(url_data.decode('utf-8')), sep = '\t')


In [31]:
""" 3. Using tweet ids from tweet_df and the Twitter API, load information such as "retweet count" and "favorite count" and any other interesting data items 
and store the entire set of data relating to a tweet in a JSON file called "tweet_json.txt"; then read this file line by line into a data frame and include 
tweet id, retweet count, favorite count and any other data items of interest
"""

twitter_keys = load_yaml_file(twitter_keys_yaml_file)
consumer_key = twitter_keys['consumer_key']
consumer_secret = twitter_keys['consumer_secret']
access_token = twitter_keys['access_token']
access_token_secret = twitter_keys['access_token_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

error_tweets = {}
tweet_offset = 0
tweet_batch = 100
tweets_remaining = 0

# Delete the file first so that each run can add content fresh
if os.path.isfile(tweet_json_file):
    os.remove(tweet_json_file)

with open(tweet_json_file, mode = 'a', encoding = 'utf-8') as f:
    while tweet_batch:
        tweet_ids = tweets_df[tweet_offset:tweet_offset + tweet_batch]['tweet_id']
        for tweet_id in tweet_ids:
            try:
                status = api.get_status(tweet_id)
                f.write(json.dumps(status._json) + '\n')

            except tweepy.TweepError as e:
                error_tweets[status.id] = e
            
        tweets_remaining = tweet_count - tweet_offset
        tweet_batch = tweet_batch if tweet_batch < tweets_remaining else tweets_remaining
        tweet_offset += tweet_batch
        # print ('%s %d %s %d' % ('Tweets Remaining: ', tweets_remaining, 'Tweet Offset: ', tweet_offset))
        # time.sleep(1 * 5)


Rate limit reached. Sleeping for: 764
Rate limit reached. Sleeping for: 762


In [33]:
with open(tweet_json_file, mode = 'r', encoding = 'utf-8') as f:
    print('%s %d %s %d' % ('Valid tweet count: ', len(f.readlines()), 'Error tweet count: ', len(error_tweets)))

Valid tweet count:  2339 Error tweet count:  17


In [0]:
# Read "tweet_json.txt" file line by line into a data frame and include tweet id, retweet count, favorite count and any other data items of interest

columns = ['tweet_id', 'text', 'retweet_count', 'favorite_count', 'retweeted', 'favorited', 'media_url']
tweets_mini_df = pd.DataFrame(columns = columns)

index = 0
with open(tweet_json_file, mode = 'r', encoding = 'utf-8') as f:
    for tweet in f:
        tweet_json = json.loads(tweet)
        tweet_id = tweet_json['id']
        text = tweet_json['text']
        retweet_count = tweet_json['retweet_count']
        favorite_count = tweet_json['favorite_count']
        retweeted = tweet_json['retweeted']
        favorited = tweet_json['favorited']
        try:
            media_url = tweet_json['entities']['media'][0]['media_url_https']
        except KeyError as e:
            media_url = ''

        tweets_mini_df.loc[index] = pd.Series({'tweet_id': tweet_id, 
                                        'text': text, 
                                        'retweet_count': retweet_count, 
                                        'favorite_count': favorite_count,
                                        'retweeted': retweeted,
                                        'favorited': favorited,
                                        'media_url': media_url})
        index +=1


### Assessing data

Key Points

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.

* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.

* Cleaning includes merging individual pieces of data according to the rules of tidy data.

* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This unique rating system is a big part of the popularity of WeRateDogs.

* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.


#### twitter-archive-enhanced.csv

In [35]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [47]:
tweets_df.query('name == "None"').sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1656,683357973142474752,NaN,NaN,2016-01-02 18:43:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...","""Have a seat, son. There are some things we ne...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/683357973...,10,10,None,None,None,None,None
1447,696488710901260288,NaN,NaN,2016-02-08 00:20:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",12/10 revolutionary af https://t.co/zKzq4nIY86,NaN,NaN,NaN,https://twitter.com/dog_rates/status/696488710...,12,10,None,None,None,None,None
1466,694342028726001664,NaN,NaN,2016-02-02 02:10:14 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",It's okay pup. This happens every time I liste...,NaN,NaN,NaN,https://vine.co/v/iJWKejYdLlh,11,10,None,None,None,None,None
1255,710609963652087808,NaN,NaN,2016-03-17 23:33:12 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",I've watched this a million times and you prob...,NaN,NaN,NaN,https://vine.co/v/idaTpwH5TgU,12,10,None,None,None,None,None
742,780476555013349377,NaN,NaN,2016-09-26 18:38:05 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Patreon: Well. @dog_rates is on Patreon. \...,7.804657e+17,1.228326e+09,2016-09-26 17:55:00 +0000,"https://www.patreon.com/WeRateDogs,https://twi...",12,10,None,None,None,None,None


In [46]:
tweets_df.isnull().sum()
tweets_df.query('name == "None"').shape

(745, 17)

#### image_predictions.tsv

In [38]:
img_pred_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [44]:
img_pred_df.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1407,770093767776997377,https://pbs.twimg.com/media/CkjMx99UoAM2B1a.jpg,1,golden_retriever,0.843799,True,Labrador_retriever,0.052956,True,kelpie,0.035711,True
177,669214165781868544,https://pbs.twimg.com/media/CUmGu7-UcAA0r3O.jpg,1,minivan,0.435396,False,police_van,0.310143,False,minibus,0.068201,False
1086,718631497683582976,https://pbs.twimg.com/media/CfkXiX6W4AAmICF.jpg,1,Pomeranian,0.993718,True,Pekinese,0.003611,True,Persian_cat,0.000525,False
1707,817777686764523521,https://pbs.twimg.com/ext_tw_video_thumb/81777...,1,curly-coated_retriever,0.733256,True,flat-coated_retriever,0.214145,True,Irish_water_spaniel,0.029769,True
1187,739485634323156992,https://pbs.twimg.com/media/CkMuP7SWkAAD-2R.jpg,2,Walker_hound,0.640256,True,English_foxhound,0.229799,True,beagle,0.037754,True


In [49]:
list(img_pred_df)

['tweet_id',
 'jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog']

#### tweets_mini_df

In [0]:
tweets_mini_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2339 entries, 0 to 2338
Data columns (total 7 columns):
tweet_id          2339 non-null object
text              2339 non-null object
retweet_count     2339 non-null object
favorite_count    2339 non-null object
retweeted         2339 non-null object
favorited         2339 non-null object
media_url         2339 non-null object
dtypes: object(7)
memory usage: 146.2+ KB


In [48]:
tweets_mini_df.sample(5)
tweets_mini_df.query('tweet_id == 683357973142474752')

,tweet_id,text,retweet_count,favorite_count,retweeted,favorited,media_url
1640,683357973142474752,"""Have a seat, son. There are some things we ne...",995,3053,False,False,https://pbs.twimg.com/media/CXvGbWeWMAcRbyJ.jpg


In [0]:
tweets_mini_df.isnull().sum()

tweet_id          0
text              0
retweet_count     0
favorite_count    0
retweeted         0
favorited         0
media_url         0
dtype: int64

#### Cleaning data

#### 1. twitter-archive-enhanced.csv
##### 1.1 Data Quality Issues:

1. There are 181 rows in this dataset that are retweets. I have identified them based non-null values in the "retweeted_status_id" column. These need to be removed from the dataset as we only want original tweets (1.1.1)

2. Remove the following columns from the dataset. I don't think that we need these columns for our analysis (1.1.2)

    *  in_reply_to_status_id
    *  in_reply_to_user_id
    *  retweeted_status_id
    *  retweeted_status_user_id
    *  retweeted_status_timestamp
    *  expanded_urls

3. The "timestamp" column is represented as a string and needs to be converted to datetime (1.1.3)
4. Dog stages are not fully recorded and many of them contain null values or values labeled as "None". We need to scan the tweet text and see if we can extract these values for each row (1.1.4)
5. Drop all tweets with timestamps greater than 8/1/2017 as per the project requirements (1.1.5)
6. *** Not Addressed *** There are 745 rows in this dataset where the name is marked as "None". I am not addressing this issue right now since I don't have sufficient data to obtain this information (1.1.6)

##### 1.2 Tidiness Issues:
1. Dog stages are represented in separate columns as a sparse-array matrix which is wasteful and also not scalable in case we identify new stages. These need to be converted to a single column titled "stage". If they are not mutually exclusive (meaning, if there are multiple tweets about the same dog at different points in time where their stages change), we can put them into a separate dataframe for normalization purposes (1.2.1)


#### 2. image-predictions.tsv

##### 2.1 Tidiness Issues:

1. In the case of this dataset, we will deal with the tidiness issues first before we deal with data quality issues. There is a specific reason for it - there are three breed predictions in this dataset with a percentage of confidence against each and a boolean flag indicating whether the image prediction is actually canine or not. That makes it a total of 9 columns. I would like to reduce them to three columns to begin with: breed, prediction_confidence, is_canine. Addressing this tidiness issue will make it easier to fix the data quality issues identified below. We will begin with separating breed predictions to its own dataframe so that we can work on the data quality issues later (2.1.1)

##### 2.2 Data Quality Issues:

1. Once we address the tidiness issue above and spin off the breed predictions to their own dataframe, we will eliminate all predictions where the "is_canine" flag is false. We will then pick the prediction with the higher percentage of confidence and retain them as the only breed predictions (assuming that each dog can be classified into a single breed) and drop the ones with lower prediction confidence scores (2.2.1)
2. We can then merge this dataset back to the original dataset (2.2.2)
3. After merging, we will drop the following columns as they are no longer needed (2.2.3): 
    *  jpg_url (we have obtained the media_url using the Twitter API and we will retain that in case it needs to be used)
    *  img_num (don't see how this information will be useful)
    *  p1
    *  p1_conf
    *  p1_dog
    *  p2
    *  p2_conf
    *  p2_dog
    *  p3
    *  p3_conf
    *  p3_dog




#### 3. tweet_mini_df

##### 3.1 Data Quality Issues
1. We need to drop all the tweets in this dataset that were originally marked as retweets in the "twitter-archive-enhanced" dataset as we are only interested in original tweets (3.1.1)


#### 4. Tidiness Issues: combining data from all the datasets

##### 4.1 Tidiness

1. Having addressed the data quality and tidiness issues at the level of individual datasets, we can now merge the data in these datasets into one. Since the data granularity in each of these datasets is at the "tweet_id" level, we will use it to combine the data into a single dataset and persist it to a file (4.1.1)

##### 4.2 Data Quality

1. We will use "inner joins" to merge these datasets thus eliminating any rows for whcih no valid information is available online. For example, there are 17 tweet ids in the original "twitter-archive-enhanced" dataset for which the Twitter API returned errors and these will be eliminated during the merging process. (4.2.1)

In [0]:
# There are 181 rows in this dataset that are retweets. This is based on the fact that the column "retweeted_status_id" is not null. 
# This column points to the original tweet which this has retweeted. These need to be removed from the dataset as we only want original ratings

tweets_df[tweets_df['retweeted_status_id'].notna()].shape[0]

181

In [0]:
# "timestamp" is represented as a string and should be converted to datetime
tweets_df['timestamp'].head(5)

0    2017-08-01 16:23:56 +0000
1    2017-08-01 00:17:27 +0000
2    2017-07-31 00:18:03 +0000
3    2017-07-30 15:58:51 +0000
4    2017-07-29 16:00:24 +0000
Name: timestamp, dtype: object

In [0]:
# Dog stages ('doggo', 'floofer', 'pupper', 'puppo') are not fully recorded in the respective columns with most values having been set to "None"
# This needs to be corrected

tweets_df[tweets_df['text'].str.contains("pupper")][['tweet_id', 'pupper', 'text']]

,tweet_id,pupper,text
29,886366144734445568,pupper,This is Roscoe. Another pupper fallen victim t...
49,882762694511734784,pupper,This is Gus. He's quite the cheeky pupper. Alr...
54,881666595344535552,None,This is Gary. He couldn't miss this puppertuni...
56,881536004380872706,pupper,Here is a pupper approaching maximum borkdrive...
78,877611172832227328,pupper,RT @rachel2195: @dog_rates the boyfriend and h...
82,876838120628539392,pupper,This is Ginger. She's having a ruff Monday. To...
92,874296783580663808,pupper,This is Jed. He may be the fanciest pupper in ...
97,873337748698140672,pupper,RT @dog_rates: This is Sierra. She's one preci...
98,873213775632977920,pupper,This is Sierra. She's one precious pupper. Abs...
107,871762521631449091,pupper,This is Rover. As part of pupper protocol he h...
